In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Step 1: Load the dataset
df = pd.read_csv('/mnt/data/USA_Housing.csv')

# Step 2: Display basic information
print("First 5 rows of the dataset:")
print(df.head())
print("\nSummary statistics:")
print(df.describe())
print("\nDataset info:")
print(df.info())

# Step 3: Check for missing values
print("\nMissing values in dataset:")
print(df.isnull().sum())

# Step 4: Visualize the dataset
sns.pairplot(df)
plt.show()

# Correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Feature Correlation Heatmap')
plt.show()

# Step 5: Outlier Detection using IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df_cleaned = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

print("\nData after outlier removal:")
print(df_cleaned.shape)

# Step 6: Define features and target variable
X = df_cleaned.drop(columns=['Address', 'Price'])
y = df_cleaned['Price']

# Step 7: Feature Engineering - Log Transformation of Target Variable
df_cleaned['Log_Price'] = np.log(df_cleaned['Price'])
y = df_cleaned['Log_Price']  # Change target to the log-transformed value

# Step 8: Check linear separability (scatter plot)
plt.figure(figsize=(6, 5))
sns.scatterplot(x=X.iloc[:, 0], y=y)
plt.xlabel('First Feature')
plt.ylabel('Log(Target Price)')
plt.title('Checking Linearity')
plt.show()

# Step 9: Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 10: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 11: Initialize Ridge Regression model with hyperparameter tuning
ridge_model = Ridge()

# Step 12: Hyperparameter tuning using GridSearchCV
parameters = {'alpha': [0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(ridge_model, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(f"Best parameters from GridSearchCV: {grid_search.best_params_}")

# Step 13: Train the model with the best hyperparameter
best_ridge_model = grid_search.best_estimator_
best_ridge_model.fit(X_train, y_train)

# Step 14: Make predictions
y_pred_ridge = best_ridge_model.predict(X_test)

# Step 15: Evaluate the model
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("\nRidge Regression Model Evaluation:")
print(f"Mean Absolute Error: {mae_ridge}")
print(f"Mean Squared Error: {mse_ridge}")
print(f"Root Mean Squared Error: {rmse_ridge}")
print(f"R^2 Score: {r2_ridge}")

# Step 16: Visualize actual vs predicted values
plt.figure(figsize=(6, 5))
sns.scatterplot(x=y_test, y=y_pred_ridge)
plt.xlabel('Actual Prices (Log-transformed)')
plt.ylabel('Predicted Prices (Log-transformed)')
plt.title('Actual vs Predicted Prices')
plt.show()

# Step 17: Check for multicollinearity using VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print("\nVariance Inflation Factor (VIF) for features:")
print(vif_data)
